## Import

In [1]:
!pip install pytorchvideo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 264.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=3e228b842810f3b8747d35acfc143b7bd8005367a613665da5f18d723693dc44
  Stored in directory: /root/.cache/pip/wheels/e0/85/41/42153f93cd5715669b2fd7765b25cfca52c7277ad5a6762112
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61429 sha256=2b2cc1d9346c51a681d55899659cbf648eac5db93701620855f4a31c18f587ee
  Stored in directory:

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import f1_score,accuracy_score
from pytorchvideo.transforms import Normalize

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/Colab Notebooks/competition/dacon/[230206]제1회 코스포 x 데이콘 자동차 충돌 분석 AI경진대회(채용 연계형)'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/competition/dacon/[230206]제1회 코스포 x 데이콘 자동차 충돌 분석 AI경진대회(채용 연계형)


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [5]:
CFG = {
    'VIDEO_LENGTH':50,
    'IMG_SIZE':256,
    'EPOCHS':15,
    'LEARNING_RATE':5e-5,
    'BATCH_SIZE':8,
    'SEED':41
}

## Fixed RandomSeed

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(CFG['SEED'])

## Data Load

In [7]:
df = pd.read_csv('.data/train.csv')

In [8]:
weather_snowy =  [22,33,79,118,137,162,201,209,219,233,235,259,262,270,291,293,360,
367,381,420,424,429,437,454,498,506,568,600,609,611,613,626,
683,708,734,763,785,797,819,857,893,927,944,953,965,
976,981,988,1013,1038,1040,1042,1064,1073,1085,1088,1092,
1110,1149,1155,1171,1181,1186,1196,1216,
1246,1247,1268,1300,1304,1305,1306,1319,1342,1350,1372,1376,1397,1405,
1417,1456,1460,1462,1466,1480,1493,1509,1536,1547,1557,1577,
1582,1600,1618,1629,1653,1655,1665,1670,1717,1729,1738,1747,
1762,1823,1842,1857,1891,1897,1907,1911,1916,1919,1948,1951,1956,1981,
1990,1992,2002,2022,2066,2086,2090,2107,2116,2131,2146,2151,2182,2237,2266,2269,
2272,2303,2317,2343,2408,2423,2453,2473,2516,2577,2597,2606,2611,2614,
2616,2637,2643,2653,2665,2674,2679]

weather_rainy =  [13,19,28,43,45,46,52,63,91,94,95,102,116,122,134,136,138,
171,183,192,198,256,304,312,326,333,345,348,361,368,370,371,398,
405,417,435,436,438,446,451,459,460,468,502,510,527,531,532,534,540,
542,544,552,557,579,601,607,615,627,638,640,650,689,735,764,808,823,
826,827,837,849,854,859,864,881,883,898,907,910,934,945,949,980,983,992,1020,1028,
1032,1062,1071,1079,1109,1129,1132,1150,1170,1190,1238,1242,1261,1269,1283,1286,
1316,1322,1323,1325,1344,1349,1351,1359,1393,1396,1403,1411,1448,1474,1476,1478,
1484,1534,1562,1576,1591,1594,1599,1602,1616,1685,1702,1710,1751,1767,1790,1804,
1809,1814,1826,1834,1841,1846,1870,1879,1927,1929,1978,1988,1993,2000,2057,2064,
2075,2087,2093,2101,2127,2200,2214,2244,2250,2264,2267,2294,2311,2355,2384,
2424,2427,2443,2456,2464,2481,2483,2492,2518,2531,2568,2580,2601,2646,2648,2649,
2652,2660,2690,2696]

timing_night = [3,5,8,12,14,22,24,26,28,29,32,34,36,39,40,46,51,52,53,57,71,72,73,
76,82,86,87,89,90,91,92,93,94,96,100,101,102,103,110,112,116,119,121,130,133,
134,135,138,141,147,150,154,155,158,162,163,166,169,176,178,180,181,183,184,
185,187,188,191,192,193,199,200,201,209,214,218,224,229,231,236,237,239,240,
245,249,250,251,256,260,262,264,269,273,275,278,286,293,298,302,312,315,319,
320,322,324,326,328,333,335,336,344,351,354,360,362,363,365,368,370,371,386,388,
390,391,392,393,395,398,405,406,408,414,417,421,423,427,434,436,438,442,444,446,
447,448,449,450,455,458,460,464,467,472,473,476,477,484,485,490,502,503,504,510,
512,514,516,522,524,525,527,530,531,532,533,534,536,538,543,547,550,557,560,561,
574,580,584,589,591,592,593,595,598,600,601,603,604,609,613,614,615,619,627,633,
634,635,640,643,646,647,650,651,656,659,660,664,669,670,671,672,678,687,689,692,
694,695,696,698,701,702,707,708,709,710,715,717,719,723,724,727,730,731,732,733,
734,735,736,738,739,741,745,757,758,776,784,785,788,794,798,806,808,811,813,816,
822,824,833,835,836,837,841,843,847,849,854,857,859,876,884,887,888,891,893,894,
895,900,905,907,908,909,911,912,918,921,922,925,930,932,934,935,939,942,950,951,
952,958,963,966,967,972,974,977,982,984,985,988,989,990,993,1010,1012,1013,1019,
1021,1022,1025,1026,1028,1029,1036,1040,1044,1051,1057,1058,1059,1063,1065,
1069,1072,1075,1078,1079,1080,1085,1091,1093,1097,1099,1106,1108,1109,1111,1112,
1114,1121,1122,1124,1129,1130,1132,1137,1139,1140,1142,1143,1144,1145,1146,
1150,1154,1155,1157,1158,1170,1171,1174,1176,1183,1185,1188,1190,1194,1195,1196,
1205,1206,1211,1212,1215,1222,1225,1226,1229,1231,1233,1235,1236,1241,1242,1243,
1247,1248,1261,1262,1264,1275,1276,1278,1279,1282,1294,1296,1300,1304,1313,1314,
1315,1316,1323,1325,1326,1331,1336,1341,1344,1347,1351,1352,1353,1355,1357,1359,
1361,1364,1367,1368,1370,1378,1379,1382,1383,1385,1392,1393,1397,1400,1401,1404,
1405,1408,1411,1412,1413,1417,1426,1429,1432,1434,1443,1447,1448,1449,1450,
1453,1457,1458,1459,1462,1466,1469,1471,1473,1474,1478,1487,1489,1493,1495,1496,
1500,1501,1504,1507,1509,1515,1522,1524,1529,1530,1538,1539,1545,1547,1549,1556,
1558,1564,1570,1571,1573,1576,1579,1583,1585,1589,1593,1601,1604,1610,1615,1616,
1618,1621,1622,1627,1632,1638,1639,1640,1643,1645,1648,1655,1664,1665,1667,1675,
1683,1686,1687,1688,1689,1691,1697,1700,1702,1704,1706,1708,1710,1715,1717,1722,
1732,1733,1735,1737,1743,1745,1750,1751,1752,1757,1760,1763,1767,1769,1774,1776,
1780,1782,1787,1790,1791,1792,1797,1798,1799,1805,1809,1810,1811,1818,1821,1822,
1826,1831,1832,1837,1844,1846,1852,1855,1859,1868,1869,1870,1871,1875,1878,1880,
1881,1882,1885,1888,1890,1892,1893,1897,1902,1905,1907,1910,1913,1915,1918,1919,
1921,1924,1926,1927,1928,1934,1937,1942,1943,1948,1950,1954,1957,1958,1959,1960,
1961,1962,1966,1982,1986,1988,1989,1993,1997,2000,2001,2002,2004,2006,2008,2016,
2018,2020,2023,2026,2027,2029,2030,2031,2035,2036,2037,2038,2044,2045,2049,2050,
2052,2054,2060,2075,2077,2079,2086,2087,2091,2093,2095,2097,2098,2099,2100,2102,
2103,2105,2107,2109,2110,2111,2113,2115,2116,2117,2120,2123,2126,2127,2129,2132,
2133,2151,2152,2168,2169,2174,2177,2179,2183,2184,2185,2194,2201,2202,2205,2208,
2209,2210,2211,2216,2219,2226,2229,2238,2245,2247,2250,2255,2256,2261,2264,2268,
2269,2270,2272,2274,2276,2279,2280,2282,2288,2296,2300,2303,2307,2308,2309,2310,
2315,2316,2319,2320,2321,2325,2326,2329,2330,2331,2334,2335,2336,2339,2342,2344,
2346,2351,2361,2362,2364,2365,2371,2372,2373,2374,2376,2378,2380,2384,2385,2386,
2387,2389,2392,2395,2398,2411,2413,2414,2415,2417,2424,2426,2430,2436,2446,2452,
2454,2457,2458,2461,2465,2470,2471,2477,2481,2482,2483,2485,2487,2489,2493,2494,
2497,2499,2500,2501,2502,2504,2506,2509,2511,2512,2518,2524,2527,2533,2535,2536,
2539,2545,2546,2550,2551,2552,2559,2562,2565,2568,2569,2581,2583,2584,2593,2601,
2602,2611,2621,2623,2624,2625,2633,2637,2638,2641,2643,2644,2648,2649,2653,2664,
2671,2674,2676,2677,2680,2683,2687,2691,2696]


In [9]:
right_label = {4:3,56:9,123:1,149:3,234:9,238:10,242:3,263:3,306:5,332:9,341:5,401:7,402:11,507:7,588:6,621:9,628:8,
630:4,632:4,652:3,693:4,720:9,728:3,778:3,800:1,856:4,861:3,877:10,889:3,892:9,896:3,917:9,920:3,970:9,1060:10,
1081:4,1086:10,1098:4,1193:5,1227:8,1251:3,1263:4,1287:9,1492:6,1505:4,1506:7,1581:11,1586:6,1605:3,
1637:8,1654:3,1656:3,1681:0,1698:3,1727:5,1728:8,1838:0,1848:5,1874:4,1917:10,1923:9,
1949:9,1955:3,2147:7,2193:4,2236:10,2239:9,2249:3,2391:10,2541:8,2570:5,2571:12,2595:4,2615:9,2654:4}

noise_label= [104,124,146,182,387,486,607,679,825,961,1113,1184,1207,1220,1318,1421,1475,1499,1526,1537,1565,1612,1903,2289,2292,2445,2508,2657,2678,2695]

weather_noise = [8,13,19,27,34,41,63,84,85,98,117,125,130,135,155,175,176,188,226,243,265,299,315,319,350,361,363,365,392,
418,422,427,458,469,477,579,581,584,629,641,671,686,697,701,721,732,746,772,818,823,828,853,865,868,895,938,947,954,
968,974,1010,1011,1025,1031,1035,1039,1054,1066,1096,1139,1200,1206,1210,1212,1217,1225,1229,1278,1307,1317,1325,
1331,1366,1401,1442,1469,1477,1489,1496,1498,1517,1539,1543,1558,1568,1593,1621,1628,1640,1658,1664,1671,
1691,1715,1722,1742,1743,1760,1781,1818,1822,1902,1905,1915,1942,1960,1997,2004,2005,2026,2077,2081,2134,
2156,2209,2228,2234,2235,2256,2270,2271,2277,2307,2315,2326,2333,2348,2387,2454,2470,2475,2487,2498,2504,2531,
2536,2545,2551,2562,2569,2572,2586,2594,2624,2625,2638,2660,2672,2677]


for i,v in right_label.items():
  df['label'].loc[i] = v

df['video_path']=df['video_path'].apply(lambda x : '.data'+x[1:])

new_labels = []
for i,(id,path,label) in enumerate(df.values):
  if label==0:
    crash_ego_involve = 0
    weather = 0
    timing = 0
  else:
    crash_ego_involve = 1+(label-1)//6
    weather = (label-1)%6//2
    timing = (label-1)%2
  new_labels.append([crash_ego_involve,weather,timing])
df[['crash_ego_involve','weather','timing']] = new_labels

df['weather'].loc[weather_snowy] = 1
df['weather'].loc[weather_rainy] = 2
df['timing'].loc[timing_night] = 1

df.drop(noise_label,inplace=True)
df.drop(weather_noise,inplace=True)
df.reset_index(drop=True,inplace=True)

In [10]:
df.head()

,sample_id,video_path,label,crash_ego_involve,weather,timing
0,TRAIN_0000,.data/train/TRAIN_0000.mp4,7,2,0,0
1,TRAIN_0001,.data/train/TRAIN_0001.mp4,7,2,0,0
2,TRAIN_0002,.data/train/TRAIN_0002.mp4,0,0,0,0
3,TRAIN_0003,.data/train/TRAIN_0003.mp4,0,0,0,1
4,TRAIN_0004,.data/train/TRAIN_0004.mp4,3,1,1,0


## Train / Validation Split

In [11]:
# train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])
train = df
len(train)

2509

## CustomDataset

In [12]:
mean = (0.45, 0.45, 0.45)
std = (0.225, 0.225, 0.225)

class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.resizing = False
        self.norm = Normalize(mean, std)

    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])[:,2:,:,:]
        low_frames = frames[:,[i*4+3 for i in range(12)],:,:]

        if self.label_list is not None:
            label = self.label_list[index]
            return low_frames, frames, label
        else:
            return low_frames, frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            img = img / 255.
            frames.append(img)
        return self.norm(torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2))

In [13]:
train_dataset = CustomDataset(train['video_path'].values, train[['crash_ego_involve','weather','timing']].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=2)
# val_dataset = CustomDataset(val['video_path'].values, val[['crash_ego_involve','weather','timing']].values)
# val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

## Model Define

In [14]:
model_name = "slowfast_r50"
slowfast_r50 = torch.hub.load("facebookresearch/pytorchvideo", model=model_name,pretrained=True)
slowfast_r50_layers = slowfast_r50.blocks

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /root/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth


  0%|          | 0.00/264M [00:00<?, ?B/s]

In [15]:
class SlowFast(nn.Module):
    def __init__(self,img_size=256):
        super(SlowFast, self).__init__()
        self.pool_size = img_size//32-1

        self.layer_1 = slowfast_r50_layers[0]
        self.layer_2 = slowfast_r50_layers[1]
        self.layer_3 = slowfast_r50_layers[2]
        self.layer_4 = slowfast_r50_layers[3]
        self.layer_5 = slowfast_r50_layers[4]
        self.fast_pool = nn.AvgPool3d(kernel_size=(12, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))
        self.slow_pool = nn.AvgPool3d(kernel_size=(48, self.pool_size, self.pool_size), stride=(1, 1, 1), padding=(0, 0, 0))

        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.crash_fc = nn.Linear(2304,3)
        self.weather_fc = nn.Linear(2304,3)
        self.timing_fc = nn.Linear(2304,2)
        for para in self.layer_1.parameters():
           para.requires_grad = False
        for para in self.layer_2.parameters():
           para.requires_grad = False

    def forward(self, x):
        batch_size = x[0].size(0)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)

        x_1 = self.fast_pool(x[0])
        x_2 = self.slow_pool(x[1])
        x = torch.concat([x_1,x_2],axis=1)
        x = self.avgpool(x).view(batch_size,-1)

        crash = self.crash_fc(x)
        weather = self.weather_fc(x)
        timing = self.timing_fc(x)
        return crash,weather,timing

## Train

## Run!!

In [16]:
crash_samples = train.groupby('crash_ego_involve',as_index=False)['sample_id'].count()['sample_id'].values
weather_samples = train.groupby('weather',as_index=False)['sample_id'].count()['sample_id'].values
timing_samples = train.groupby('timing',as_index=False)['sample_id'].count()['sample_id'].values
origin_samples = train.groupby('label',as_index=False)['sample_id'].count()['sample_id'].values

def focal_loss(labels, logits, alpha, gamma):
    BCLoss = F.binary_cross_entropy_with_logits(input = logits, target = labels,reduction = "none")
    if gamma == 0.0:
        modulator = 1.0
    else:
        modulator = torch.exp(-gamma * labels * logits - gamma * torch.log(1 + 
            torch.exp(-1.0 * logits)))
    loss = modulator * BCLoss

    weighted_loss = alpha * loss
    focal_loss = torch.sum(weighted_loss)

    focal_loss /= torch.sum(labels)
    return focal_loss

def convert_to_label(crash,weather,timing):
  labels = 1 + (crash-1)*6 + weather*2 + timing
  labels[crash==0] = 0
  return labels

def CB_loss(logits, labels, origin_labels, samples_per_cls, no_of_classes, loss_type='sigmoid', beta=0.999, beta_2 = 0.95, gamma=2):

    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes
    labels_one_hot = F.one_hot(labels.long(), no_of_classes).float()

    weights = torch.tensor(weights).float()
    weights = weights.unsqueeze(0)
    weights = (weights.repeat(labels_one_hot.shape[0],1).to(device)) * labels_one_hot
    weights = weights.sum(1)
    weights = weights.unsqueeze(1)
    weights = weights.repeat(1,no_of_classes)
    
    ## 원본 데이터 weights
    effective_num_origin = 1.0 - np.power(beta_2, origin_samples)
    weights_origin = (1.0 - beta_2) / np.array(effective_num_origin)
    weights_origin = weights_origin / np.sum(weights_origin) * 13
    labels_one_hot_origin = F.one_hot(origin_labels.long(), 13).float()

    weights_origin = torch.tensor(weights_origin).float()
    weights_origin = weights_origin.unsqueeze(0)
    weights_origin = (weights_origin.repeat(labels_one_hot_origin.shape[0],1).to(device)) * labels_one_hot_origin
    weights_origin = weights_origin.sum(1)
    weights_origin = weights_origin.unsqueeze(1)
    weights_origin = weights_origin.repeat(1,no_of_classes)
    
    weights = weights*weights_origin
    
    if loss_type == "focal":
        cb_loss = focal_loss(labels_one_hot, logits, weights, gamma)
    elif loss_type == "sigmoid":
        cb_loss = F.binary_cross_entropy_with_logits(input = logits,target = labels_one_hot, weight = weights)
    return cb_loss

In [17]:
def slowfast_loss(crash,weather,timing,labels):
  origin_labels = convert_to_label(labels[:,0],labels[:,1],labels[:,2])

  crash_loss = CB_loss(crash,labels[:,0],origin_labels,crash_samples,3,loss_type='focal')
  weather_loss = CB_loss(weather,labels[:,1],origin_labels,weather_samples,3,loss_type='focal')
  timing_loss = CB_loss(timing,labels[:,2],origin_labels,timing_samples,2,loss_type='focal')

  return crash_loss + weather_loss + timing_loss

In [18]:
torch.cuda.empty_cache()
file_name = 'submit'

model = SlowFast(img_size=CFG['IMG_SIZE'])
model.eval()
best_model = None
optimizer = optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"],weight_decay=0.02)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[12], gamma=0.1)
model = model.to(device)

In [19]:
best_val_score = 0
os.makedirs('./.final',exist_ok=True)

for epoch in range(1, CFG['EPOCHS']+1):
    model.train()
    train_loss = []
    crash_preds, weather_preds, timing_preds = [], [], []
    crash_trues, weather_trues, timing_trues = [], [], []
    for fast_videos, videos, labels in tqdm(iter(train_loader)):
        fast_videos = fast_videos.to(device)
        videos = videos.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        crash,weather,timing = model([fast_videos,videos])
        loss = slowfast_loss(crash,weather,timing,labels)

        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

        crash_preds += crash.argmax(1).detach().cpu().numpy().tolist()
        weather_preds += weather.argmax(1).detach().cpu().numpy().tolist()
        timing_preds += timing.argmax(1).detach().cpu().numpy().tolist()

        crash_trues += labels[:,0].detach().cpu().numpy().tolist()
        weather_trues += labels[:,1].detach().cpu().numpy().tolist()
        timing_trues += labels[:,2].detach().cpu().numpy().tolist()
    scheduler.step()
    _train_loss = np.mean(train_loss)
    
    origin_trues = convert_to_label(np.array(crash_trues) , np.array(weather_trues) , np.array(timing_trues))
    origin_preds = convert_to_label(np.array(crash_preds) , np.array(weather_preds) , np.array(timing_preds))
    train_accuracy = accuracy_score(origin_trues, origin_preds)
    train_score = f1_score(origin_trues,origin_preds, average='macro')

    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Train Acc : [{train_accuracy:.5f}]] Train F1 : [{train_score :.5f}]\n')

    # model.eval()
    # val_loss = []
    # crash_preds, weather_preds, timing_preds = [], [], []
    # crash_trues, weather_trues, timing_trues = [], [], []

    # with torch.no_grad():
    #     for fast_videos, videos, labels in tqdm(iter(val_loader)):
    #         fast_videos = fast_videos.to(device)
    #         videos = videos.to(device)
    #         labels = labels.to(device)
            
    #         crash,weather,timing = model([fast_videos,videos])
            
    #         loss = slowfast_loss(crash,weather,timing,labels)
            
    #         val_loss.append(loss.item())

    #         crash_preds += crash.argmax(1).detach().cpu().numpy().tolist()
    #         weather_preds += weather.argmax(1).detach().cpu().numpy().tolist()
    #         timing_preds += timing.argmax(1).detach().cpu().numpy().tolist()

    #         crash_trues += labels[:,0].detach().cpu().numpy().tolist()
    #         weather_trues += labels[:,1].detach().cpu().numpy().tolist()
    #         timing_trues += labels[:,2].detach().cpu().numpy().tolist()
          
    #     _val_loss = np.mean(val_loss)

    # origin_trues = convert_to_label(np.array(crash_trues) , np.array(weather_trues) , np.array(timing_trues))
    # origin_preds = convert_to_label(np.array(crash_preds) , np.array(weather_preds) , np.array(timing_preds))
    # _val_each_score = f1_score(origin_trues,origin_preds, average=None)
    # _val_score = f1_score(origin_trues,origin_preds, average='macro')

    # print('Val each F1 : ')
    # print('0 \t 1 \t 2 \t 3 \t 4 \t 5 \t 6 \t 7 \t 8 \t 9 \t 10 \t 11 \t 12')
    # print('--------------------------------------------------------------------------------------------------------')
    # print(" ".join(list(map(lambda x:format(x,'.5f'),_val_each_score))))
    # print(f'Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}] \n\n')
    # if best_val_score < _val_score:
    #       best_val_score = _val_score
    #       best_model = model
    #       torch.save(best_model.state_dict(),f'.model/{file_name}_{epoch}.pt')
    #       print('*********** best model saved ***********')  

  0%|          | 0/314 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.28539] Train Acc : [0.78637]] Train F1 : [0.41116]



  0%|          | 0/314 [00:00<?, ?it/s]

Epoch [2], Train Loss : [0.11339] Train Acc : [0.92228]] Train F1 : [0.71334]



  0%|          | 0/314 [00:00<?, ?it/s]

...

Epoch [13], Train Loss : [0.00299] Train Acc : [0.99960]] Train F1 : [0.99974]



  0%|          | 0/314 [00:00<?, ?it/s]

Epoch [14], Train Loss : [0.00223] Train Acc : [0.99960]] Train F1 : [0.99802]



  0%|          | 0/314 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.00183] Train Acc : [1.00000]] Train F1 : [1.00000]



In [20]:
torch.save(model.state_dict(),f'.final/{file_name}.pt')

## Inference

In [21]:
torch.cuda.empty_cache()

In [22]:
test = pd.read_csv('.data/test.csv')

In [23]:
test['video_path']=test['video_path'].apply(lambda x : '.data'+x[1:])

In [24]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

In [25]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    crash_preds,weather_preds,timing_preds = [], [], []
    with torch.no_grad():
        for fast_videos,videos in tqdm(iter(test_loader)):
            fast_videos = fast_videos.to(device)
            videos = videos.to(device)

            crash,weather,timing = model([fast_videos,videos])

            crash_preds += crash.argmax(1).detach().cpu().numpy().tolist()
            weather_preds += weather.argmax(1).detach().cpu().numpy().tolist()
            timing_preds += timing.argmax(1).detach().cpu().numpy().tolist()
    return crash_preds, weather_preds, timing_preds

In [26]:
model = SlowFast(img_size=CFG['IMG_SIZE'])
print(file_name)
model.load_state_dict(torch.load(f'.final/{file_name}.pt'))
model.eval()

crash_preds, weather_preds, timing_preds = inference(model, test_loader, device)

submit


  0%|          | 0/225 [00:00<?, ?it/s]

## Submission

In [27]:
submit = pd.read_csv('.data/sample_submission.csv')

In [28]:
submit['crash'] = crash_preds
submit['weather'] = weather_preds
submit['timing'] = timing_preds

In [29]:
submit['label'] = 1 + (submit['crash']-1)*6 + submit['weather']*2 + submit['timing']
submit['label'][submit['crash']==0] = 0
submit[['sample_id','label']].head(10)

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
5,TEST_0005,1
6,TEST_0006,0
7,TEST_0007,0
8,TEST_0008,0
9,TEST_0009,0


In [30]:
os.makedirs('./.final_submit',exist_ok=True)
submit[['sample_id','label']].to_csv(f'.final_submit/{file_name}.csv', index=False)